In [1]:
import os, glob, re

import pandas as pd
import numpy as np

from pathlib import Path
from sklearn.model_selection import train_test_split

In [2]:
from transferlearning import ConvBaseSearch

# Glob all jpg-ending files and regex the category out of each
p = Path("data/*/*")
paths_images = glob.glob(str(p) + "*jpg")
cat = [re.search("\d+\.", i).group().strip(".") for i in paths_images]

df = pd.DataFrame(list(zip(paths_images,cat)), columns=["path", "cat"])
df = df[df['cat'].map(df['cat'].value_counts()) > 150]

target_col, path_col = "cat", "path"

pretrained_models = ["vgg16", "vgg19"] #Models that should be tested

Using TensorFlow backend.


In [3]:
# Initialize ConvBaseSearch and pass the model names you want to test
cbs = ConvBaseSearch(pretrained_models)

In [4]:
# Fit the data to the pretrained models. Only use sample_size proportion of whole dataset.
cbs.fit(df, path_col, target_col, sample_size=0.02, data_augmentation=True)

Found 163 validated image filenames belonging to 12 classes.
Found 41 validated image filenames belonging to 12 classes.
Fitting  vgg16
Epoch 1/1


UnknownError:  Failed to get convolution algorithm. This is probably because cuDNN failed to initialize, so try looking to see if a warning log message was printed above.
	 [[node vgg16/block1_conv1/convolution (defined at /home/d4ve/.local/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:3009) ]] [Op:__inference_keras_scratch_graph_1617]

Function call stack:
keras_scratch_graph


In [5]:
# Get the best score 
cbs.best_score

0.2222222238779068

In [6]:
# Get the best model
best_base = cbs.best_base
print("Name:", best_base.name)
print("Keras model:", best_base.model)

Name: vgg16
Keras model: <keras.engine.training.Model object at 0x000002C074E10F88>
